In [2]:
import pandas as pd
import numpy as np
import glob
#list custom names for CSVs
df_names = ['customer', 'inventory', 'products', 'sales', 'stores']

#get all CSV files
files = glob.glob("Pandas Retail Store/*.csv") 
files.sort() #sort alphabetically to match df_names

#load CSVs into a dictionary with custom names
dfs = {name: pd.read_csv(file) for name, file in zip(df_names, files)}

#check everything is loaded correctly and show first 3 rows
for name, df in dfs.items():
    print(f"==={name}===")
    print(df.head(3))
    

===customer===
   customer_id first_name last_name                  email           phone  \
0            1      James     Smith  james.smith@gmail.com    555-123-4567   
1            2       Mary   Johnson                    NaN    212.555.6789   
2            3       John  Williams    jwilliams@yahoo.com  (555) 987-6543   

        address         city       state    zip_code registration_date  
0   123 Main St     New York          NY       10001        2021-03-15  
1  456 Park Ave     New York          NY       10022        2020-11-02  
2  789 Broadway  Los Angeles  California  90001-1234        2021-05-20  
===inventory===
   inventory_id  store_id  product_id  quantity_in_stock last_restock_date  \
0             1         1           1                 25        2022-10-15   
1             2         1           5                 42        2022-11-02   
2             3         1          10                 18        2022-09-30   

   reorder_level  
0           10.0  
1           1

In [11]:
#Print summary statistics for products, sales, inventory
for name in ['products', 'sales', 'inventory']:
    print(f"\n===Summary Statistics: {name}===")
    print(dfs[name].describe())


===Summary Statistics: products===
       product_id        price         cost     weight
count   30.000000    30.000000    30.000000  28.000000
mean    15.500000   191.724000   203.159000   3.835714
std      8.803408   452.483576   278.924138   5.693236
min      1.000000 -1599.990000     1.800000   0.200000
25%      8.250000    46.990000    28.375000   0.500000
50%     15.500000   134.990000    82.500000   1.150000
75%     22.750000   322.490000   236.250000   3.750000
max     30.000000  1299.990000  1100.000000  25.000000

===Summary Statistics: sales===
         sale_id   store_id  customer_id  product_id   quantity        total
count  40.000000  39.000000    40.000000   40.000000  40.000000    40.000000
mean   20.500000   6.487179    14.075000   13.500000   1.625000   352.296500
std    11.690452   4.235637     8.303189    8.857852   0.952392   377.625776
min     1.000000   1.000000     1.000000    1.000000   1.000000    19.980000
25%    10.750000   3.000000     6.750000    5.75000

In [18]:
#Check for missing values in all dataframes
for name, df in dfs.items():
    print(f"\n===Missing Values in {name} ===")
    print(df.isnull().sum())


===Missing Values in customer ===
customer_id          0
first_name           0
last_name            0
email                3
phone                2
address              0
city                 0
state                0
zip_code             1
registration_date    3
dtype: int64

===Missing Values in inventory ===
inventory_id         0
store_id             0
product_id           0
quantity_in_stock    0
last_restock_date    9
reorder_level        6
dtype: int64

===Missing Values in products ===
product_id      0
product_name    0
category        0
subcategory     0
brand           0
price           0
cost            0
weight          2
dtype: int64

===Missing Values in sales ===
sale_id           0
date              1
store_id          1
customer_id       0
product_id        0
quantity          0
total             0
payment_method    1
dtype: int64

===Missing Values in stores ===
store_id        0
store_name      0
address         0
city            0
state           0
zip_code       

In [20]:
#Display data types for each column in all dataframes
for name, df in dfs.items():
    print(f"\n===Data Types of {name}===")
    print(df.dtypes)


===Data Types of customer===
customer_id           int64
first_name           object
last_name            object
email                object
phone                object
address              object
city                 object
state                object
zip_code             object
registration_date    object
dtype: object

===Data Types of inventory===
inventory_id           int64
store_id               int64
product_id             int64
quantity_in_stock      int64
last_restock_date     object
reorder_level        float64
dtype: object

===Data Types of products===
product_id        int64
product_name     object
category         object
subcategory      object
brand            object
price           float64
cost            float64
weight          float64
dtype: object

===Data Types of sales===
sale_id             int64
date               object
store_id          float64
customer_id         int64
product_id          int64
quantity            int64
total             float64
payment_meth

In [52]:
#Show how many unique products are in the product catalog
print(f"Number of unique products is:", dfs['products']['product_id'].nunique())

Number of unique products is: 30


In [57]:
#Top 5 most expensive products
expensive_products = dfs['products'].nlargest(5, 'price')[['product_name', 'price']]
print(expensive_products)

          product_name    price
3          Dell XPS 13  1299.99
0      Apple iPhone 13   899.99
1   Samsung Galaxy S21   799.99
19    Dyson V11 Vacuum   599.99
26  Sony PlayStation 5   499.99


In [58]:
#Check for the store w/ greatest sq footage
max_space = dfs['stores'][['store_name', 'size_sqft']].max()
print(max_space)

store_name    Seattle City Center
size_sqft                 55000.0
dtype: object


In [61]:
#Check for distribution of customers per state
customers_per_state = dfs['customer'].groupby('state')['customer_id'].count()
print(customers_per_state)

state
AZ                1
CA                3
CO                1
California        1
FL                1
IL                1
IN                1
KY                1
MA                1
MD                1
MI                1
MO                1
Maryland          1
NM                1
NV                1
NY                2
North Carolina    1
OH                1
Oregon            1
PA                1
TN                1
TX                4
Texas             1
WI                1
Washington        1
Name: customer_id, dtype: int64


In [80]:
#Fill numeric columns with mean value of that column
for name, df in dfs.items():
    #Select numeric columns
    numeric_columns = df.select_dtypes(include='number').columns
    #Fill null values w/ mean
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
    #Update df in dictionary
    dfs[name] = df


In [76]:
#Run this before categorical columns as date columns have object data type
for name, df in dfs.items():
    #Identify date columns   
    date_columns = [col for col in df.columns if 'date' in col]
    
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce') #Convert to datetime
        df[col] = df[col].ffill().bfill() #Fill missing value

    #Update df in dictionary
    dfs[name] = df

In [79]:
#Fill categorical columns with most frequent value
for name, df in dfs.items():
    #Select categorical columns
    cat_columns = df.select_dtypes(include='object').columns
    #fill with mode value
    df[cat_columns] = df[cat_columns].fillna(df[cat_columns].mode())
    #Update df in dictionary
    dfs[name] = df

In [85]:
#Check missing values were filled
for name, df in dfs.items():
    print(f"\n===Missing Values in {name} ===")
    print(df.isnull().sum())


===Missing Values in customer ===
customer_id          0
first_name           0
last_name            0
email                1
phone                0
address              0
city                 0
state                0
zip_code             1
registration_date    0
dtype: int64

===Missing Values in inventory ===
inventory_id         0
store_id             0
product_id           0
quantity_in_stock    0
last_restock_date    0
reorder_level        0
dtype: int64

===Missing Values in products ===
product_id      0
product_name    0
category        0
subcategory     0
brand           0
price           0
cost            0
weight          0
dtype: int64

===Missing Values in sales ===
sale_id           0
date              0
store_id          0
customer_id       0
product_id        0
quantity          0
total             0
payment_method    1
dtype: int64

===Missing Values in stores ===
store_id        0
store_name      0
address         0
city            0
state           0
zip_code       

In [87]:
#Check for full duplicates across all rows
for name, df in dfs.items():
    num_duplicates = df.duplicated().sum()
    print(f"\nDataset: {name}")
    print(f"Number of duplicates are: {num_duplicates}")


Dataset: customer
Number of duplicates are: 0

Dataset: inventory
Number of duplicates are: 0

Dataset: products
Number of duplicates are: 0

Dataset: sales
Number of duplicates are: 0

Dataset: stores
Number of duplicates are: 0



=== Duplicates in customer dataset ===
Full row duplicates: 0

=== Duplicates in inventory dataset ===
Full row duplicates: 0
Duplicates based on 'inventory_id': 0

=== Duplicates in products dataset ===
Full row duplicates: 0
Duplicates based on 'product_id': 0

=== Duplicates in sales dataset ===
Full row duplicates: 0
Duplicates based on 'sale_id': 0

=== Duplicates in stores dataset ===
Full row duplicates: 0
Duplicates based on 'store_id': 0
